In [3]:
#create 1d cnn model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN1d(nn.Module):
    #create a 1d cnn regression model
    #the ouput is a target value of 60 minutes later, is a scalar
    def __init__(self, input_len, input_dim, kernel_size, layer_num, hidden_size,
                  dropout=0.5, batch_norm=True):
        #input shape: (batch_size, input_len, input_dim)
        super(CNN1d, self).__init__()
        self.input_len = input_len
        self.input_dim = input_dim
        self.kernel_size = kernel_size
        self.layer_num = layer_num
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.batch_norm = batch_norm
        self.convs = nn.ModuleList([nn.Conv1d(self.input_dim, self.hidden_size, self.kernel_size)])
        self.bns = nn.ModuleList([nn.BatchNorm1d(self.hidden_size)])
        for i in range(self.layer_num - 1):
            self.convs.append(nn.Conv1d(self.hidden_size, self.hidden_size, self.kernel_size))
            self.bns.append(nn.BatchNorm1d(self.hidden_size))
        self.fc1 = nn.Linear(self.hidden_size, 1)
        self.dropout = nn.Dropout(self.dropout)
        self.init_weights()

    def init_weights(self):
        #initialize weights
        for conv in self.convs:
            torch.nn.init.xavier_uniform_(conv.weight)
        torch.nn.init.xavier_uniform_(self.fc1.weight)

    def forward(self, x):
        #forward pass
        #input shape: (batch_size, input_len, input_dim)
        #output shape: (batch_size, 1)
        x = x.transpose(1, 2)
        for i, conv in enumerate(self.convs):
            if self.batch_norm:
                x = F.relu(self.bns[i](conv(x)))
            else:
                x = F.relu(conv(x))
            x = self.dropout(x)
        x = self.fc1(x[:, :, -1])
        return x
    
    def predict(self, x):
        #predict the target value
        return self.forward(x)
    
    def loss(self, x, y):
        #calculate loss
        return F.mse_loss(self.forward(x), y)




In [4]:
#positional encoding
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class PositionalEncoding(nn.Module):
    #add positional encoding to the input
    def __init__(self, input_len, input_dim, dropout=0.5):
        super(PositionalEncoding, self).__init__()
        self.input_len = input_len
        self.input_dim = input_dim
        self.dropout = nn.Dropout(dropout)
        self.pe = torch.zeros(self.input_len, self.input_dim)
        position = torch.arange(0, self.input_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.input_dim, 2).float() * (-math.log(10000.0) / self.input_dim))
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = self.pe.unsqueeze(0)
        self.pe.requires_grad = False

    def forward(self, x):
        #forward pass
        #input shape: (batch_size, input_len, input_dim)
        #output shape: (batch_size, input_len, input_dim)
        x = x + self.pe
        return self.dropout(x)


In [58]:
class Attention(nn.Module):
    #create MULTIHEADATTENTION model
    #the ouput is a target value of 60 minutes later, is a scalar
    def __init__(self, input_len, input_dim, hidden_size, num_heads, dropout=0.5):
        super(Attention, self).__init__()
        self.input_len = input_len
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.q_linear = nn.Linear(self.input_dim, self.hidden_size)
        self.v_linear = nn.Linear(self.input_dim, self.hidden_size)
        self.k_linear = nn.Linear(self.input_dim, self.hidden_size)
        self.multihead_attn = nn.MultiheadAttention(self.hidden_size, self.num_heads)
        self.out = nn.Linear(self.hidden_size, 1)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        #use positional encoding
        x = PositionalEncoding(self.input_len, self.input_dim)(x)
        #forward pass
        bs = x.size(0)
        q = self.q_linear(x)
        k = self.k_linear(x)
        v = self.v_linear(x)
        attn_output, _ = self.multihead_attn(q, k, v)
        attn_output = self.dropout(attn_output)
        output = self.out(attn_output[:, -1, :])
        return output

    def predict(self, x):
        #predict the target value
        return self.forward(x)
    
    def loss(self, x, y):
        #calculate loss
        return F.mse_loss(self.forward(x), y)
    


In [99]:

num_samples = 1000
#test the model
X = torch.randn(num_samples, 10)
Y = torch.randn(num_samples, 1)


In [100]:
#data loader
day = torch.randint(0, 30, (num_samples, 1))
class MyDataset(torch.utils.data.Dataset):

    def __init__(self, X, y, day, input_len=128):
        #the input data is a 1d array, indicate the minute of the day
        self.X = X
        self.y = y
        self.day = day
        self.input_len = input_len

    def __getitem__(self, index):
        #output previous self.input_len minutes data and target value
        #if the there is no enough data in the same day, pad with 0
        d = self.day[index]
        start = index - self.input_len
        if start < 0:
            start = 0
        if self.day[start] != d:
            while self.day[start] != d:
                start += 1
            #pad with 0 before start
        if index - start < self.input_len:
            x = torch.zeros(self.input_len, 10)
            x[self.input_len - index + start: self.input_len] = self.X[start: index].clone()
        else:
            x = self.X[start: index]
            
        y = self.y[index].clone()
        if x.shape != (self.input_len, 10):
            print(x.shape, index, start)
        return x, y
    def __len__(self):
        return len(self.X)
    
mydataset = MyDataset(X, Y, day, input_len=128)
train_index = range(0, int(num_samples * 0.8))
test_index = range(0, int(num_samples * 0.8))
#test_index = range(int(num_samples * 0.8), num_samples)

train_dataset = torch.utils.data.Subset(mydataset, train_index)
test_dataset = torch.utils.data.Subset(mydataset, test_index)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)



In [108]:
#train the model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Attention(input_dim=10, input_len=128, hidden_size=64, num_heads=8).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
for epoch in range(10):
    for i, data in enumerate(train_loader):
        x, y = data
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        loss = model.loss(x, y)
        loss.backward()
        optimizer.step()
    scheduler.step()
    print("epoch: {}, loss: {}".format(epoch, loss.item()))

    #evaluate the model
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            loss = model.loss(x, y)
            print("test loss: {}".format(loss.item()))
            break
    model.train()

pred = torch.zeros(len(test_dataset), 1)
with torch.no_grad():
    for i, data in enumerate(test_loader):
        x, y = data
        x = x.to(device)
        y = y.to(device)
        pred[i * 128: i * 128 + len(x)] = model.predict(x)
        print(y)
    


epoch: 0, loss: 0.8228365182876587
test loss: 0.8946669101715088
tensor([[ 1.0340e-01],
        [ 2.7850e-01],
        [-1.4524e+00],
        [ 4.4976e-01],
        [-1.6339e+00],
        [-1.3247e+00],
        [-1.0558e+00],
        [ 4.0196e-01],
        [-7.0728e-01],
        [ 1.2954e+00],
        [-4.4962e-01],
        [-6.5616e-01],
        [-8.1838e-01],
        [-1.1353e+00],
        [-2.5619e-01],
        [-1.0760e+00],
        [ 7.8851e-01],
        [ 1.0763e+00],
        [ 1.3571e+00],
        [ 4.4242e-01],
        [-4.4572e-01],
        [ 2.8043e-01],
        [ 3.5877e-01],
        [ 1.8783e+00],
        [ 1.5051e+00],
        [-3.4021e-01],
        [-5.4949e-01],
        [ 1.7317e-01],
        [ 1.0465e+00],
        [-1.4213e+00],
        [ 3.6202e-01],
        [-1.1025e-01],
        [ 7.2494e-01],
        [ 3.5854e-01],
        [-5.0771e-04],
        [ 1.9347e+00],
        [ 1.5889e+00],
        [-2.3667e+00],
        [-3.6950e-02],
        [-8.2248e-01],
        [ 7.164

In [106]:
(pred - Y[test_index]).abs().mean()

tensor(0.7947)

In [92]:
pred - y[test_index]

IndexError: index 800 is out of bounds for dimension 0 with size 72

In [88]:
# def scores(pred, target, days):
#     pred_group, ground_group, corr = groupbyday(pred, target, days)
#     sharpes = sharpeyear(corr, list(pred_group.keys()))
#     return corr.mean(), sharpes
    
# def fitmodel(model, training_data, test_data, training_target, test_target, training_days, test_days):
#     for train, test, train_target, test_target, train_day, test_day in zip(training_data, test_data, training_target, test_target, training_days, test_days):
#         model.fit(train, train_target)
#         pred_train = model.predict(train)
#         pred_test = model.predict(test)
#         pnl_train, sharpe_train = scores(pred_train, train_target,train_day)
#         pnl_test, sharpe_test = scores(pred_test, test_target, test_day)
#         return pnl_train, sharpe_train, pnl_test, sharpe_test

In [63]:
import numpy as np
model = Attention(input_dim=10, input_len=128, hidden_size=64, num_heads=8)
def trainmodel(model, data, target, days, train_index, test_index, epochs=100):
    #get dataset
    mydataset = MyDataset(data, target, days)
    train_index = range(0, int(num_samples * 0.8))
    test_index = range(int(num_samples * 0.8), num_samples)

    train_dataset = torch.utils.data.Subset(mydataset, train_index)
    test_dataset = torch.utils.data.Subset(mydataset, test_index)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)
    #train the model
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    for epoch in range(epochs):
        for i, data in enumerate(train_loader):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            loss = model.loss(x, y)
            loss.backward()
            optimizer.step()
        scheduler.step()
        print("epoch: {}, loss: {}".format(epoch, loss.item()))

        #evaluate the model
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(test_loader):
                x, y = data
                x = x.to(device)
                y = y.to(device)
                loss = model.loss(x, y)
                print("test loss: {}".format(loss.item()))
    
    pred = np.zeros(len(test_dataset), 1)
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            pred[i * 128: i * 128 + len(x)] = model.predict(x)
    return pred


In [64]:
trainmodel(model, X, y, day, train_index, test_index, epochs=10)

epoch: 0, loss: 0.7170489430427551
test loss: 1.0776116847991943
epoch: 1, loss: 1.1976138353347778
test loss: 1.0752530097961426
epoch: 2, loss: 1.0374078750610352
test loss: 1.1087355613708496
epoch: 3, loss: 1.279557466506958
test loss: 1.078538179397583
epoch: 4, loss: 0.9948725700378418
test loss: 1.057780385017395
epoch: 5, loss: 0.9333169460296631
test loss: 1.0650285482406616
epoch: 6, loss: 0.7389782667160034
test loss: 1.0728360414505005
epoch: 7, loss: 1.4094113111495972
test loss: 1.0735621452331543
epoch: 8, loss: 0.7442904114723206
test loss: 1.056931734085083
epoch: 9, loss: 0.6093248128890991
test loss: 1.112714409828186


ValueError: only one element tensors can be converted to Python scalars